In [11]:
# Connecting to the Database in mySQL
import mysql.connector
from mysql.connector import Error

try:
    # Establish the connection
    connection = mysql.connector.connect(
        host='localhost',           # Hostname (assuming a local database)
        user='root',       # MySQL username
        password='your_password',   # MySQL password (please don't steal lol)
        database='ECOMMERCE' # Name of the database you created
    )

    if connection.is_connected():
        db_info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_info)
        cursor = connection.cursor(buffered=True)
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)



Error while connecting to MySQL 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)


In [12]:
# Trigger #1
createTrigger= """
CREATE TRIGGER update_inventory_after_transaction_insert
AFTER INSERT ON TRANSACTION_DETAIL
FOR EACH ROW
BEGIN
    DECLARE inventory_count INT;
    SELECT Available INTO inventory_count FROM INVENTORY WHERE ProductID = NEW.ProductID;
    IF inventory_count IS NOT NULL THEN
        UPDATE INVENTORY SET Available = inventory_count - NEW.Quantity WHERE ProductID = NEW.ProductID;
    END IF;
END;
"""

try: 
    cursor.execute(createTrigger)
    print("Trigger 'UpdateInventoryForPlacedOrders")
except mysql.connector.Error as error:
    print("Error creating trigger:", error)


Trigger 'UpdateInventoryForPlacedOrders


In [13]:
InventoryListQuery = """
SELECT * 
FROM INVENTORY;
"""

def InventoryList():
    try:
        cursor.execute(InventoryListQuery)
        
        print("Inventory: ")
        print("InventoryID\tProductID\tAvailable")
        for product in cursor: 
            print(f"{product[0]}\t\t{product[1]}\t\t{product[2]}")
        
    except mysql.connector.Error as error: 
        print("Error fetching inventory: ", error)


In [14]:
# Procedure #1
createProductProcedure = """
CREATE PROCEDURE AddProductAndInventory(
    IN product_name VARCHAR(100),
    IN description MEDIUMTEXT,
    IN price DECIMAL(15,2),
    IN initial_quantity INT,
    IN product_type VARCHAR(20),
    IN battery_type VARCHAR(100),
    IN collectible_condition VARCHAR(100)
)
BEGIN
    DECLARE product_id INT;

    -- Insert into PRODUCT table
    INSERT INTO PRODUCT (Product_name, Description, Price)
    VALUES (product_name, description, price);

    -- Get the last inserted ProductID
    SET product_id = LAST_INSERT_ID();

    -- Insert into INVENTORY table
    INSERT INTO INVENTORY (ProductID, Available)
    VALUES (product_id, initial_quantity);

    -- Insert into subclass table based on product type
    IF product_type = 'electronic' THEN
        INSERT INTO ELECTRONIC (ProductID, BatteryType)
        VALUES (product_id, battery_type);
    ELSEIF product_type = 'collectible' THEN
        INSERT INTO COLLECTIBLE (ProductID, CollectibleCondition)
        VALUES (product_id, collectible_condition);
    END IF;
END
"""
try: 
    cursor.execute(createProductProcedure)
    print("Created AddProductAndInventory Procedure Succefully")
except mysql.connector.Error as error: 
    print("Error when trying to make Procedure: ", error)

def createProduct():
    
    try: 
        ProductName = input("What is the Products Name? ")
        description = input("Description of product (leave empty if not needed.): ")
        Quantity = input("How many Products are you adding? ")
        price = input("What is the Price of the Product? ")
        productType = input("What is the product's Type Electronic/Collectible? (Leave empty if neither.)")
        productType = productType.lower()
        batteryType = None
        collectibleCondition = None
        
        if productType == 'electronic':
            batteryType = input("What is the battery type? ")
        elif productType == 'collectible':
            collectibleCondition = input("What is the collectible's condition?")
    
        cursor.callproc("AddProductAndInventory", (ProductName, description, price, Quantity, productType, batteryType, collectibleCondition))
        connection.commit()
        print("Added Product Succesfully")
    except mysql.connector.Error as error: 
        print("Error when trying to add product. Aborting:  ", error)
    

Error when trying to make Procedure:  1304 (42000): PROCEDURE AddProductAndInventory already exists


In [15]:
# Procedure #2
editInvProcedure = """
CREATE PROCEDURE UpdateInventory(
    IN productId INT,
    IN quantityChange INT
)
BEGIN
    UPDATE INVENTORY
    SET Available = Available + quantityChange
    WHERE ProductID = productId;
END
"""

try:
    cursor.execute(editInvProcedure)
    print("Stored procedure 'UpdateInventory' created successfully.")
except mysql.connector.Error as error:
    print("Error creating stored procedure:", error)

def editInventoryProd():
    try:
        productID = int(input("Enter the Product ID: "))
        change = int(input("Enter the quantity change (+ for increase, - for decrease): "))
        cursor.callproc("UpdateInventory", (productID, change))
        connection.commit()
        print("Inventory updated successfully!")
    except ValueError:
        print("Invalid input. Please enter valid Product ID and quantity change.")
    except mysql.connector.Error as error:
        print("Error updating inventory:", error)
        connection.rollback()
    

Stored procedure 'UpdateInventory' created successfully.


In [16]:


def deleteProd():
    

SyntaxError: incomplete input (1842076069.py, line 2)

In [ ]:


def getMostPop():
    

In [ ]:


def getLeastPop():
    

In [ ]:


def getAfkUsers():
    

In [ ]:
def createNewTransaction():
    

In [ ]:

# try:

#         # SQL command to drop the stored procedure
#         drop_procedure_query = f"DROP PROCEDURE IF EXISTS AddProductAndInventory"

#         # Execute the SQL command to drop the stored procedure
#         cursor.execute(drop_procedure_query)
#         print(f"Stored procedure AddProductAndInventory deleted successfully.")
# except mysql.connector.Error as error:
#         print(f"Error deleting stored procedure: {error}")
    

Stored procedure AddProductAndInventory deleted successfully.


In [17]:
#Console menu function
def mainMenu():
    while True: 
        print("\nConsole Menu")
        print("0. Exit")
        print("1. Products Currently in Inventory.")
        print("2. Create New Product.")
        print("3. Modify a Product in Inventory")
        print("4. Delete a Product from Inventory.")
        print("5. Get Most Popular Products from the given time range.")
        print("6. Get Least Popular Products from the given time range.")
        print("7. Send Promotional Emails to unactive Users.")
        print("8. Create Transaction")
        print("9. TBD")
        
        choice = input("Enter your choice: ")
        match choice: 
            case '0': 
                print("\nExiting.")
                return
            case '1':
                print("\nProducts currently in Inventory are: ")
                InventoryList()
            case '2':
                print("\nAdding Product")
                createProduct()
            case '3':
                print("\nEditing Inventory.")
                editInventoryProd()
            case '4': 
                print("")
                deleteProd()
            case '5':
                print("")
                getMostPop()
            case '6': 
                print("")
                getLeastPop()
            case '7': 
                print("")
                getAfkUsers()
            case '8': 
                print("")
            case '9':
                print("")
            case _:
                print("\nPlease enter a digit 0-9.")
            
mainMenu() 



Console Menu
0. Exit
1. Products Currently in Inventory.
2. Create New Product.
3. Modify a Product in Inventory
4. Delete a Product from Inventory.
5. Get Most Popular Products from the given time range.
6. Get Least Popular Products from the given time range.
7. Send Promotional Emails to unactive Users.
8. Create Transaction
9. TBD

Products currently in Inventory are: 
Inventory: 
InventoryID	ProductID	Available
1		1		12
990000		10000		10
990001		10001		46
990002		10002		1
990003		10003		4
990004		10004		39
990005		10005		84
990006		20000		9
990007		20001		10
990008		20002		13
990009		30000		5
990010		30001		3
990011		30002		72

Console Menu
0. Exit
1. Products Currently in Inventory.
2. Create New Product.
3. Modify a Product in Inventory
4. Delete a Product from Inventory.
5. Get Most Popular Products from the given time range.
6. Get Least Popular Products from the given time range.
7. Send Promotional Emails to unactive Users.
8. Create Transaction
9. TBD

Products currently i

In [ ]:
# Closing the connection to the database 

if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

MySQL connection is closed
